In [1]:
import matplotlib
matplotlib.use('Agg')
import sys
# sys.path.append("/home/ec2-user/anaconda3/external/fastai/")
sys.path.append('/scratch/arka/miniconda3/external/fastai/')


from tqdm import tqdm
tqdm.monitor_interval = 0

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import json
import pandas as pd
from sklearn.metrics import *

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
PATH = Path("/scratch/arka/miniconda3/external/fastai/courses/dl2/data/cifar10/")
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [4]:
def total_model_params(summary):
    total_params = 0
    for key,value in summary.items():
        total_params+=int(summary[key]['nb_params'])
    print("Total parameters in the model :"+str(total_params))

In [5]:
bs=128
sz=32

In [6]:
class depthwise_block(nn.Module):
    def __init__(self, in_c, out_c, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_c)
        self.conv1 = nn.Conv2d(in_c, in_c, kernel_size=3, 
                               stride=stride, padding=1,
                               groups=in_c, bias=False)
        self.conv2 = nn.Conv2d(in_c, out_c, kernel_size=1,
                              stride=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_c)
    
    def forward(self, inp):
        out = F.relu(self.bn1(self.conv1(inp)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out

In [7]:
class mblnetv1(nn.Module):
    def __init__(self, block, inc_list, inc_scale, num_blocks_list, stride_list, num_classes):
        super().__init__()
        self.num_blocks = len(num_blocks_list)
        inc_list1 = [o//inc_scale for o in inc_list]
        self.in_planes = inc_list1[0]
        self.conv1 = nn.Conv2d(3, self.in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_planes)
        
        lyrs = []
        for inc, nb, strl in zip(inc_list1[1:], num_blocks_list, stride_list):
            lyrs.append(self._make_layer(block, inc, nb, strl))
            
        self.lyrs = nn.Sequential(*lyrs)
        self.linear = nn.Linear(inc_list1[-1], num_classes)
        
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, inp):
        out = F.relu(self.bn1(self.conv1(inp)))
        out = self.lyrs(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return F.log_softmax(out, dim=-1)


In [ ]:
# md_mbl = mblnetv1(depthwise_block, 
#               inc_list=[64, 64, 128, 256], 
#               inc_scale = 1, 
#               num_blocks_list=[2, 2, 2], 
#               stride_list=[1, 2, 2], 
#               num_classes=10)

In [8]:
md_mbl2 = mblnetv1(depthwise_block, 
              inc_list=[32, 64, 128, 128, 256, 256, 512, 512, 1024, 1024], 
              inc_scale = 1, 
              num_blocks_list=[1, 1, 1, 1, 1, 1, 4, 1, 1], 
              stride_list=[1, 2, 1, 2, 1, 2, 1, 2, 1], 
              num_classes=10)

In [ ]:
!ls {PATH / 'train'}

In [9]:
data = get_data(sz, bs)

In [10]:
learn = ConvLearner.from_model_data(md_mbl2, data)

In [11]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 32, 32]),
                           ('output_shape', [-1, 32, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 864)])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 32, 32, 32]),
                           ('output_shape', [-1, 32, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 64)])),
             ('Conv2d-3',
              OrderedDict([('input_shape', [-1, 32, 32, 32]),
                           ('output_shape', [-1, 32, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 288)])),
             ('BatchNorm2d-4',
              OrderedDict([('input_shape', [-1, 32, 32, 32]),
                           ('output_shape', [-1, 32, 32, 32]),
                           ('trainable', True),
                           ('nb_params', 64)])

In [12]:
total_model_params(learn.summary())

Total parameters in the model :2948426


In [13]:
learn.fit(1e-2, 1, cycle_len=10, best_save_name='mobilenetv1_n1', metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                   
    0      1.658271   1.54055    0.4264    
    1      1.219879   1.13346    0.5998                     
    2      1.001316   1.064548   0.6269                      
    3      0.863693   0.855163   0.7005                      
    4      0.780671   0.732126   0.7411                      
    5      0.709401   0.654558   0.7681                      
    6      0.638683   0.617586   0.7819                      
    7      0.600552   0.588638   0.7917                      
    8      0.579475   0.576877   0.7994                      
    9      0.542689   0.573967   0.8005                      



[array([0.57397]), 0.8005]

In [14]:
learn.fit(1e-2, 3, cycle_len=10, best_save_name='mobilenetv1_n1', metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                    
    0      0.734268   0.865581   0.7052    
    1      0.681313   0.730509   0.7529                      
    2      0.618408   0.602464   0.7894                      
    3      0.575283   0.585223   0.7992                      
    4      0.529284   0.52204    0.8211                      
    5      0.473791   0.502569   0.826                       
    6      0.442099   0.477031   0.8367                      
    7      0.400552   0.463427   0.8427                      
    8      0.377184   0.456668   0.8453                      
    9      0.386976   0.458852   0.8467                      
    10     0.54537    0.598722   0.7977                      
    11     0.502885   0.518392   0.8222                      
    12     0.455833   0.527304   0.8213                      
    13     0.416      0.480598   0.8372                      
    14     0.375278   0.450425   0.8474                      
    15     0.350501   0.42

[array([0.38533]), 0.8759]

In [ ]:
learn.fit(1e-2, 1, cycle_len=10, best_save_name='mobilenetv1_n1', metrics=[accuracy])

In [ ]:
learn.fit(1e-2, 1, cycle_len=1, best_save_name='mobilenetv1_n1', metrics=[accuracy])

In [ ]:
learn.fit(1e-2, 1, cycle_len=10, best_save_name='mobilenetv1_n1', metrics=[accuracy])

In [15]:
learn.fit(1e-2, 2, cycle_len=10, cycle_mult=2, best_save_name='mobilenetv1_n1', metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                    
    0      0.326854   0.484623   0.8427    
    1      0.330356   0.434581   0.8569                      
    2      0.312695   0.452508   0.8568                      
    3      0.276165   0.424632   0.8631                      
    4      0.256707   0.403921   0.8708                      
    5      0.228414   0.391802   0.8735                      
    6      0.203927   0.383796   0.8794                      
    7      0.172687   0.381061   0.881                       
    8      0.172657   0.380333   0.8808                      
    9      0.159547   0.379263   0.8825                      
    10     0.318153   0.441105   0.8561                      
    11     0.299607   0.444676   0.8551                      
    12     0.29193    0.425544   0.861                       
    13     0.268321   0.454131   0.8553                      
    14     0.25692    0.422625   0.8671                      
    15     0.243094   0.40

[array([0.39567]), 0.8872]

In [16]:
learn.fit(1e-2, 2, cycle_len=10, cycle_mult=2, best_save_name='mobilenetv1_n1', metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                     
    0      0.221751   0.480329   0.8598    
    1      0.21646    0.466539   0.8594                      
    2      0.189819   0.433004   0.8682                      
    3      0.172524   0.413878   0.8761                      
    4      0.149432   0.408873   0.8791                      
    5      0.12198    0.402442   0.8826                      
    6      0.100888   0.400838   0.8858                       
    7      0.096934   0.396921   0.8888                       
    8      0.072943   0.395847   0.8884                       
    9      0.075447   0.394207   0.8883                       
    10     0.203161   0.470915   0.8638                      
    11     0.20792    0.424956   0.8729                      
    12     0.190385   0.440949   0.872                       
    13     0.170519   0.434269   0.8742                      
    14     0.167191   0.43017    0.8756                      
    15     0.154415  

[array([0.42052]), 0.895]